In [1]:
!pip3 install torch==1.4.0 -q
!pip3 install transformers==2.9.0 -q
!pip3 install pytorch_lightning==0.7.5 -q
!pip3 install gdown -q

In [2]:
# !mkdir model

In [3]:
import gdown 
import os.path

datamodel = 'model/pytorch_model.bin'
config_file = 'model/config.json'

model_location = "https://drive.google.com/uc?id=1-0BW2zWcI_XeT_NRT1V5p66YRCGkHs0F"
config_location = "https://drive.google.com/uc?id=1-09mjTtjlgjC7bZPahBsQQoKq1ZCVCVU"
if os.path.isfile(datamodel):
    ('Model was already downloaded.')
else:
    gdown.download(model_location, datamodel, True)
    gdown.download(config_location, config_file, True)

In [4]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
 
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('./model/')

model_location = "https://drive.google.com/uc?id=1-0BW2zWcI_XeT_NRT1V5p66YRCGkHs0F"
config_location = "https://drive.google.com/uc?id=1-09mjTtjlgjC7bZPahBsQQoKq1ZCVCVU"
if os.path.isfile(datamodel):
    ('Model was already downloaded.')
    else:
        gdown.download(model_location, datamodel, True)
            gdown.download(config_location, config_file, True)

In [5]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("device ", device)
model = model.to(device)

sentence = "Trump, Biden pitch dueling visions in final sprint."


text =  "paraphrase: " + sentence + " </s>"


max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=3
)

print ("\nOriginal Phrase: ")
print (sentence)
print ("\n")
print ("Paraphrased Phrase: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))


Original Phrase: 
Trump, Biden pitch dueling visions in final sprint.


Paraphrased Phrase: 
0: Trump and Biden pitch off competing visions in the final sprint.
1: Trump and Biden make dueling visions in the final sprint.
2: Trump and Biden pitch rivalry visions in the final sprint.
